# Analyzing CIA Factbook Data Using SQL
------------------------
** In this project I will use SQL to explore data from the CIA Factbook.**

The CIA factbook contains the following information about all of the countries in the world:

- Population
- Area
- Area of water
- Area of land
- Population growth
- Birth rate
- Death rate
- Migration rate

The questions that I will answer are:

- What are the highest and lowest populations and also what are the highest and lowest population growths?
- What is the average population and area of each country?
- Which countries have above average population but lower than average area size?
- Which country has the most people? Which country has the highest growth rate?
- Which countries have the highest ratios of water to land? Which countries have more water than land?
- Which countries will add the most people to their populations next year?
- Which countries have a higher death rate than birth rate?
- Which countries have the highest population/area ratio, and how does it compare to list we found in the previous screen?

**Firstly we need to connect to the database and find the name of the file that we will be using.**

In [1]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

'Connected: None@factbook.db'

In [2]:
%%sql
SELECT *
FROM sqlite_master
WHERE type='table'

Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


So the name of the file is 'facts'.

**Let's check out the first 5 rows of 'facts' to get an idea of what we are working with.**

In [3]:
%%sql
SELECT *
FROM facts
LIMIT 5

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


'Facts' is ordered by the name column which has each country in alphabetical order. The rest of the columns then give us information about each country. Now let's start answering some of our questions. 

** What are the highest and lowest populations and also what are the highest and lowest population growths?**

In [4]:
%%sql
SELECT MAX(population), MIN(population), MAX(population_growth), MIN(population_growth)
FROM facts


Done.


MAX(population),MIN(population),MAX(population_growth),MIN(population_growth)
7256490011,0,4.02,0.0


Some interesting information here. Apparantly the highest population is above 7 billion which is far too much and the lowest population is 0!

** Let's dig deeper to find which countries have given us this data.**

In [5]:
%%sql
SELECT name, Population
FROM facts
WHERE Population = (SELECT Max(population) FROM facts)
OR Population = (SELECT Min(population) FROM facts)

Done.


name,population
Antarctica,0
World,7256490011


Hmmm well Antarctica and the world aren't countries at all! Let's get rid of them and try again.

In [6]:
%%sql
SELECT name, population 
FROM facts
WHERE Population = (SELECT Max(population) FROM facts WHERE name != 'World')
OR Population = (SELECT Min (population) FROM facts WHERE name != 'Antarctica')

Done.


name,population
China,1367485388
Pitcairn Islands,48


In [7]:
%%sql
SELECT name, population_growth
FROM facts
WHERE population_growth = (SELECT MAX(population_growth) FROM facts)
OR population_growth = (SELECT MIN(population_growth) FROM facts)

Done.


name,population_growth
South Sudan,4.02
Holy See (Vatican City),0.0
Cocos (Keeling) Islands,0.0
Greenland,0.0
Pitcairn Islands,0.0


There we go. Now we can see that China has the highest population at 1.3 billion and the Pitcairn Islands have the lowest population with just just 48.

For the population growth, the fastest growing country is the South Sudan which is growing by 4% each year. The Vatican, The Cocos Islands, Greenland and The Pitcairn islands have the lowest population growth as they are all not growing at all.

** Next Question. What is the average population and area of each country?**

In [8]:
%%sql
SELECT round(AVG(population), 2) as 'Average Population', round(AVG(area), 2) as 'Average Area'
FROM facts

Done.


Average Population,Average Area
62094928.32,555093.55


The average population is just over 62 million and the average area is 555,093. Which brings us on to the next question:

**Which countries have above average population but lower than average area size?**

In [9]:
%%sql
SELECT name, population, area
FROM facts
WHERE population > (SELECT AVG(Population) FROM facts)
AND area < (SELECT AVG(area) FROM facts)

Done.


name,population,area
Bangladesh,168957745,148460
Germany,80854408,357022
Japan,126919659,377915
Philippines,100998376,300000
Thailand,67976405,513120
United Kingdom,64088222,243610
Vietnam,94348835,331210


These 7 countries have a large population but not much land for all of the people!

**Which countries have the highest ratios of water to land? Which countries have more water than land?**

In [10]:
%%sql
SELECT name, round(CAST(area_water as FLOAT) / area_land, 2) as ratio
FROM facts
ORDER BY ratio DESC
LIMIT 10

Done.


name,ratio
British Indian Ocean Territory,905.67
Virgin Islands,4.52
Puerto Rico,0.55
"Bahamas, The",0.39
Guinea-Bissau,0.28
Malawi,0.26
Netherlands,0.23
Uganda,0.22
Eritrea,0.16
Liberia,0.16


By far the country with the highest ratio of water to land is The British Indian Ocean Territory. It's area is comprised of 900 times more water than land. The only other country with more water than land are The Virgin Islands. 

**Which countries will add the most people to their populations next year?**

In [11]:
%%sql
SELECT name, population, population_growth, round(CAST(population as FLOAT) / 100 * population_growth, 2)  as People_added
FROM facts
WHERE name != 'World'
ORDER BY People_added DESC
LIMIT 10

Done.


name,population,population_growth,People_added
India,1251695584,1.22,15270686.12
China,1367485388,0.45,6153684.25
Nigeria,181562056,2.45,4448270.37
Pakistan,199085847,1.46,2906653.37
Ethiopia,99465819,2.89,2874562.17
Bangladesh,168957745,1.6,2703323.92
United States,321368864,0.78,2506677.14
Indonesia,255993674,0.92,2355141.8
"Congo, Democratic Republic of the",79375136,2.45,1944690.83
Philippines,100998376,1.61,1626073.85


India will add the most people to their population next year with an extra 15 million people!

**Which countries have a higher death rate than birth rate?**

In [12]:
%%sql
SELECT name, death_rate, birth_rate, round(CAST(death_rate as FLOAT) - birth_rate, 2) as 'ratio'
FROM facts
WHERE ratio > 0
ORDER BY ratio DESC

Done.


name,death_rate,birth_rate,ratio
Bulgaria,14.44,8.92,5.52
Serbia,13.66,9.08,4.58
Latvia,14.31,10.0,4.31
Lithuania,14.27,10.1,4.17
Ukraine,14.46,10.72,3.74
Hungary,12.73,9.16,3.57
Germany,11.42,8.47,2.95
Slovenia,11.37,8.42,2.95
Romania,11.9,9.14,2.76
Croatia,12.18,9.45,2.73


All of these countries have a higher death rate than birth rate. Bulgaria is the country with the biggest gap between the death rate and birth rate.

**Which countries have the highest population/area ratio?**

In [13]:
%%sql
SELECT name, population, area, round(CAST(population as FLOAT) / area, 2) as ratio
FROM facts
WHERE ratio >= 0
ORDER BY ratio DESC
LIMIT 100

Done.


name,population,area,ratio
Macau,592731,28,21168.96
Monaco,30535,2,15267.5
Singapore,5674472,697,8141.28
Hong Kong,7141106,1108,6445.04
Gaza Strip,1869055,360,5191.82
Gibraltar,29258,6,4876.33
Bahrain,1346613,760,1771.86
Maldives,393253,298,1319.64
Malta,413965,316,1310.02
Bermuda,70196,54,1299.93


These countries show which populations are crammed into the smallest space. As city nations it is unsuprising that Macau, Monaco, Singapore and Hong Kong sit at the top of the list.

**Thanks for reading. I hope you enjoyed this quick look through population and country size.**